# Multi-modal RAG with LangChain

## SetUp

Install the dependencies you need to run the notebook.

In [ ]:
# for linux
#%suI7do apt-get install poppler-utils tesseract-ocr libmagic-dev

# for mac
# %brew install poppler tesseract libmagic

UsageError: Line magic function `%sudo` not found.


In [ ]:
# %pip install -Uq "unstructured[all-docs]" pillow lxml pillow
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv
!pip install pdfplumber
%pip install -Uq langchain-groq langchain_huggingface
!pip install ragas[critique] datasets
!pip install -U langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 19.5 MB/s eta 0

In [ ]:
import os

# keys for the services we will use
os.environ["OPENAI_API_KEY"]="-----"
os.environ["GROQ_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["PINECONE_API_KEY"] = "pcsk_2wZScE_zP5hG6kdQWfDA6jg7FLSZGFYf6VLPeaAF8mXrX3zGJPUx9rEkTVVSCX9T8w8Tg"
os.environ["PINECONE_ENVIRONMENT"] = "us-west1-gcp"  # e.g., "us-west1-gcp"



## Extract the data
xtract the elements of the PDF that we will be able t use in the retrieval process. These elements can be: Text, Images, Tables, etc.

### Partition PDF tables, text

In [ ]:
import pdfplumber
texts=[]
tables=[]
# Open the PDF and extract pages
with pdfplumber.open('/content/Sample HI Policy.pdf') as pdf:
    for page in pdf.pages:
        texts.append(page.extract_text())
        # print(text) # Extract plain text
        if page.extract_tables():
          tables.append(page.extract_tables())
          # Extract tables

## Summarize the data

Create a summary of each element extracted from the PDF. This summary will be vectorized and used in the retrieval process.

### Text and Table summaries

We don't need a multimodal model to generate the summaries of the tables and the text. I will use open source models available on Groq.

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text that perfectly describes the table in starting 2 sentences.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

#
# Summary chain
model = ChatGroq(temperature=0, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()


In [ ]:
# Summarize extracted text
text_summaries = []
if texts:
    text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

# Summarize extracted tables
tables_html = [str(table) for table in tables]  # Convert tables to string format
table_summaries = []
if tables_html:
    table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 5})


In [ ]:
table_summaries[1]

'This table contains policy details for MR. LOKU HEMANTH, including policy number, commencement and expiry dates, plan opted, and premium information. It also includes insured person details, nominee information, and intermediary and claim administrator contact details.'

## Load data and summaries to vectorstore

### Create the vectorstore (For even better embedding performance at a cost of compute)

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Linq-AI-Research/Linq-Embed-Mistral"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# index= pc.Index("gaido-rag")
# The vectorstore to use to index the child chunks
# vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=hf)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 200.88 MiB is free. Process 55109 has 39.35 GiB memory in use. Of the allocated memory 38.95 GiB is allocated by PyTorch, and 559.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2wZScE_zP5hG6kdQWfDA6jg7FLSZGFYf6VLPeaAF8mXrX3zGJPUx9rEkTVVSCX9T8w8Tg")

index_name = "gaido-rag"



from langchain_pinecone import Pinecone

text_field = "text"
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=4096,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, hf, text_field
)


<ipython-input-10-46aff0ffa9ab>:28: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 1.0.0. Use :class:`~PineconeVectorStore` instead.
  vectorstore = Pinecone(


### Create the vectorstore (For Retrieval)

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# index= pc.Index("gaido-rag")
# The vectorstore to use to index the child chunks
# vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=hf)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2wZScE_zP5hG6kdQWfDA6jg7FLSZGFYf6VLPeaAF8mXrX3zGJPUx9rEkTVVSCX9T8w8Tg")

index_name = "gaido-rag2"



from langchain_pinecone import Pinecone

text_field = "text"
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1024,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, hf, text_field
)


<ipython-input-12-819eb57a2202>:28: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 1.0.0. Use :class:`~PineconeVectorStore` instead.
  vectorstore = Pinecone(


In [ ]:

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Load the summaries and link the to the original data

In [ ]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=summary, metadata={id_key: table_ids[i]}) for i, summary in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))



In [ ]:
print(summary_tables[1])
print(tables[1])

page_content='This table contains policy details for MR. LOKU HEMANTH, including policy number, commencement and expiry dates, plan opted, and premium information. It also includes insured person details, nominee information, and intermediary and claim administrator contact details.' metadata={'doc_id': '9dd0afc1-b5d3-474c-ab0f-ccec298c47d8', 'text': 'This table contains policy details for MR. LOKU HEMANTH, including policy number, commencement and expiry dates, plan opted, and premium information. It also includes insured person details, nominee information, and intermediary and claim administrator contact details.'}
[[['Policyholder Name: MR. LOKU HEMANTH'], ['Policyholder Address:\n1-59 NARSIMHA SAGAR MULUGU TELANGANA,\n506172,\nWARANGAL,\nTELANGANA -506172']], [['Policy Number', '33033225202502'], ['Policy Commencement Date and Time', 'From 17/02/2025 00:00'], ['Policy Expiry Date and Time', 'To 16/02/2026 23:59'], ['Plan Opted', 'Individual'], ['Policy Period', '1 Year'], ['Renewa

### Check retrieval

In [ ]:
# Retrieve
docs = retriever.invoke(
    "What is the policy start and expire date?? "
)
for doc in docs:

    print(str(doc) + "\n\n" + "-" * 80)

Policy Schedule
Policyholder Name: MR. LOKU HEMANTH Policy Number 33033225202502
Policyholder Address: Policy Commencement Date and TimeFrom 17/02/2025 00:00
Policy Expiry Date and Time To 16/02/2026 23:59
1-59 NARSIMHA SAGAR MULUGU TELANGANA,
Plan Opted Individual
506172,
Policy Period 1 Year
WARANGAL,
Renewal / Payment Due Date 16/02/2026
TELANGANA -506172
Reported claims in the policy since 0
Details of Electronic Insurance Account (eIA) inception
eIA Number None
Insurance Repository Name None
Cover Details
Name of the Insured Base Sum Sum Insured Booster Sum Insured (Base Sum Live Healthy Personal
Person(s) Insured Safeguard+ Benefit Insured + Booster Benefit + Discount % Accident opted
(in Rs.) (in Rs.) accrued (in Sum Insured Safeguard+) (in
Rs.) Rs.)
Mr. Hemanth Loku 12,50,000 1,37,750 12,50,000 26,37,750 0.00 0
.(cid:1)
Premium Details
Net Integrated Central Goods and State/UT Goods and Gross Premium (Rs.) Gross Premium (Rs.)
Premium/Taxable Goods and Service Tax (0.00 Service 

## RAG pipeline

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from base64 import b64decode


def parse_docs(docs):
    """Split base64-encoded images and texts"""
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {"images": b64, "texts": text}


def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    # Extract text context
    context_text = ""
    if docs_by_type.get("texts"):
        for text_element in docs_by_type["texts"]:
            if isinstance(text_element, list):
                # Flatten nested lists before joining
                flat_text = " ".join(
                    " ".join(map(str, sub_element)) if isinstance(sub_element, list) else str(sub_element)
                    for sub_element in text_element
                )
                context_text += flat_text + "\n"
            else:
                context_text += str(text_element) + "\n"

    # Extract table context
    context_tables = ""
    if docs_by_type.get("tables"):
        for table in docs_by_type["tables"]:
            table_str = "\n".join([" | ".join(map(str, row)) for row in table])  # Convert table rows to strings
            context_tables += f"\nTable:\n{table_str}\n"

    # Construct prompt with context (including tables)
    prompt_template = f"""
    Answer the question based only on the following context only Ground Truth Final Answer, which includes text and tables.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Be specific

    Context:
    {context_text}

    {context_tables}

    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    # If images are provided, include them
    # if docs_by_type.get("images"):
    #     for image in docs_by_type["images"]:
    #         prompt_content.append(
    #             {
    #                 "type": "image_url",
    #                 "image_url": {"url": f"data:image/jpeg;base64,{image}"},
    #             }
    #         )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatGroq(model="llama-3.3-70b-versatile")
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | ChatGroq(model="llama-3.3-70b-versatile")
        | StrOutputParser()
    )
)

In [ ]:
response = chain.invoke(
    "What is the policy commencement date?"


)

print(response)

The policy commencement date is 17/02/2025 00:00.


In [ ]:
response = chain_with_sources.invoke(
    " What is the policy commencement date?"
)

print("Response:", response['response'])

print("\n\nContext:")
for text in response['context']['texts']:
    print(text)
    # print("Page number: ", text.metadata.page_number)
    print("\n" + "-"*50 + "\n")
# for image in response['context']['images']:
#     display_base64_image(image)

Response: The policy commencement date is 17/02/2025 00:00.


Context:
Policy Schedule
Policyholder Name: MR. LOKU HEMANTH Policy Number 33033225202502
Policyholder Address: Policy Commencement Date and TimeFrom 17/02/2025 00:00
Policy Expiry Date and Time To 16/02/2026 23:59
1-59 NARSIMHA SAGAR MULUGU TELANGANA,
Plan Opted Individual
506172,
Policy Period 1 Year
WARANGAL,
Renewal / Payment Due Date 16/02/2026
TELANGANA -506172
Reported claims in the policy since 0
Details of Electronic Insurance Account (eIA) inception
eIA Number None
Insurance Repository Name None
Cover Details
Name of the Insured Base Sum Sum Insured Booster Sum Insured (Base Sum Live Healthy Personal
Person(s) Insured Safeguard+ Benefit Insured + Booster Benefit + Discount % Accident opted
(in Rs.) (in Rs.) accrued (in Sum Insured Safeguard+) (in
Rs.) Rs.)
Mr. Hemanth Loku 12,50,000 1,37,750 12,50,000 26,37,750 0.00 0
.(cid:1)
Premium Details
Net Integrated Central Goods and State/UT Goods and Gross Premium (Rs.) G

In [ ]:
queries = [
    "What is the name of the insurance product?",
    "What is the policy number mentioned in the document?",
    "What is the sum insured for the policyholder?",
    "What is the policyholder's address?",
    "When does the policy start and expire?",
    "What is the plan type chosen (Individual or Family Floater)?",
    "What is the premium amount for the policy?",
    "What taxes are applicable on the premium?",
    "Is a personal accident cover included in the policy?",
    "What are the coverage options for sum insured in this policy?",
    "Does this policy cover pre-hospitalization expenses?",
    "Does this policy cover post-hospitalization expenses?",
    "Is air ambulance service covered under this policy?",
    "What is the coverage for alternative treatments like Ayurveda or Homeopathy?",
    "What is the maximum limit for robotic surgery under modern treatments?",
    "Does this policy include home care treatment?",
    "Is domiciliary hospitalization covered?",
    "Is a second medical opinion service included in this policy?",
    "How many days of hospitalization are required for inpatient care to be covered?",
    "What is the cash benefit for shared accommodation?",
    "What are the general exclusions in this policy?",
    "Is cosmetic surgery covered under the policy?",
    "Are treatments for obesity and weight control covered?",
    "Does this policy cover maternity expenses?",
    "Is dental treatment covered under this policy?",
    "Are sexually transmitted diseases covered?",
    "What is the waiting period for pre-existing diseases?",
    "Are self-inflicted injuries covered?",
    "What is the exclusion policy for hazardous sports activities?",
    "Are treatments taken outside India covered?",
    "What is the turnaround time for cashless claim approval?",
    "How many days does it take to resolve a grievance?",
    "What is the claim submission deadline for reimbursement?",
    "What documents are required for claim submission?",
    "Where can the insured check the list of network hospitals?",
    "How can an insured person track their claim status?",
    "Is there a dedicated helpline for claim support?",
    "What is the process for availing of cashless hospitalization?",
    "Is there a list of unrecognized hospitals in the document?",
    "Can a claim be rejected if treatment is taken at an unrecognized hospital?",
    "What is the booster benefit in this policy?",
    "How does the live healthy discount work?",
    "What is the maximum increase in sum insured due to booster benefits?",
    "Is there any renewal premium discount based on health metrics?",
    "What is the safeguard+ feature in this policy?",
    "Is there an option for room type modification under this policy?",
    "Does the policy provide benefits for a health check-up?",
    "Is there an annual aggregate deductible mentioned in the policy?",
    "How does the sum insured safeguard feature work?",
    "Can the sum insured be changed during renewal?"
]

ground_truths = [
    "ReAssure",
    "33033225202502",
    "INR 26,37,750",
    "1-59 NARSIMHA SAGAR MULUGU TELANGANA, 506172, WARANGAL, TELANGANA - 506172",
    "Start: 17/02/2025 00:00 Expire: 16/02/2026 23:59",
    "Individual",
    "INR 12824.00",
    "The premium in this context is subject to Integrated Goods and Service Tax (IGST) at a rate of 18%. The tax amount is calculated as follows: Gross Premium (Rs. 12,824.00) * IGST Rate (18%) = Rs. 1,956.24.",
    "No",
    "Sum Insured Options are: 3 Lacs, 4 Lacs, 5 Lacs, 7.5 Lacs, 10 Lacs, 12.5 Lacs, 15 Lacs, 20 Lacs, 25 Lacs, 50 Lacs, 75 Lacs, 100 Lacs",
    "Yes, for 60 days",
    "Yes, for 180 days",
    "Yes",
    "Upto Sum Assured",
    "The maximum limit for robotic surgeries under 'Modern treatments' is Rs. 1 Lac, except for Robotic total radical prostatectomy, Robotic cardiac surgeries, Robotic partial nephrectomy, and Robotic surgeries for malignancies. These specific procedures do not have a limit.",
    "Yes",
    "Yes",
    "Once for any condition for which hospitalization is triggered",
    "Minimum period of 24 hours",
    "Rs. 800 per day; maximum Rs. 4,800",
    "Detailed exclusion in section 6.",
    "No, cosmetic surgery is not covered under the given policy",
    "Yes, partially covered under certain conditions",
    "Yes, with specific exclusions and waiting period",
    "No, dental treatment is not explicitly covered",
    "Yes, sexually transmitted diseases (excluding HIV/AIDS) are covered",
    "36 months",
    "No, self-inflicted injuries are not covered",
    "Excludes professional participation in hazardous/adventurous sports",
    "No",
    "Pre-authorization: 1 hour; Final authorization: within 3 hours",
    "14 Days",
    "Within 48 hours of admission or before discharge, whichever earlier",
    "Claim form, medical history, identity proof, photo, discharge certificate, banking details",
    "Official website nivabupa.com",
    "User can track their claim status on website",
    "Yes, dedicated helpline: 1860-500-8888",
    "Detailed cashless hospitalization process provided",
    "Yes",
    "Yes",
    "Cumulative bonus increases sum insured by 50%, conditions apply",
    "Discount on renewal premium based on accumulated health points",
    "Maximum increase up to 100% of Base Sum Insured",
    "Yes, renewal discount based on health points accumulated via mobile app",
    "Safeguard+ includes Claim Safeguard, Booster Benefit Safeguard, and CPI-linked Sum Insured increase",
    "Yes, with medical expenses pro-rated",
    "Yes, only on a cashless basis for diagnostic tests",
    "No",
    "CPI-linked increase in Base Sum Insured, sequence for utilization specified",
    "Yes, subject to underwriting at renewal"
]


In [ ]:
results = []
contexts = []

for query in queries:
    response = chain_with_sources.invoke(query)

    # Append the generated answer
    results.append(response['response'])

    # Extract and store all source document texts for the current query
    context_texts = response['context']['texts']
    contexts.append(context_texts)

# Example to print results and contexts for verification
for i, query in enumerate(queries):
    print(f"\nQuery: {query}")
    print(f"Response: {results[i]}")
    print("Context:")
    for text in contexts[i]:
        print(text)
        print("\n" + "-"*50 + "\n")



Query: What is the name of the insurance product?
Response: The name of the insurance product is ReAssure.
Context:
ReAssure Customer Information Sheet
Benefit Illustration
Benefit Illustration (5 Lac Sum Insured, Policy Term 1 year)
Age Coverage
of opted on
the individual
Coverage opted on
mem basis
individual basis covering Coverage opted on family
bers covering
multiple members of the floater basis with overall
insur each
family under a single Sum Insured (Only one Sum
ed member of
policy (Sum Insured is Insured is available for the
the family
available for each member entire family)
separately
of the family)
(at a single
point in
time)
Prem Sum Prem Disc Prem Sum Premiu Float Prem Sum
ium Insu ium ount, ium Insu m or er ium Insu
(Rs.) red (Rs.) if any after red Consoli disco after red
(Rs.) disco (Rs.) dated unt, disco (Rs.)
unt premiu if any unt
(Rs.) m for (Rs.)
all
membe
rs of
family
(Rs.)
Illustration 1
7,862 5,00, 7,862 786.2 7,076 5,00, 7,862.0 14,69 20,66 5,00,
18 .00 000 .

In [ ]:

from ragas import evaluate
from datasets import Dataset
# from ragas.metrics.critique import harmfulness
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
import pandas

In [ ]:
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
import pandas as pd

# Correcting the format of contexts to be a list of strings
d = {
    "question": queries,
    "answer": results,
    "contexts": [ [str(item) for item in context_list] for context_list in contexts ],  # contexts as list of strings
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)

# Evaluate using RAGAS metrics
score = evaluate(
    dataset,
    metrics=[
        faithfulness, answer_relevancy, context_precision,
        context_recall, context_entity_recall, answer_similarity,
        answer_correctness
    ]
)

# Convert scores to pandas DataFrame and save to CSV
score_df = score.to_pandas()
score_df.to_csv("EvaluationScores.csv", encoding="utf-8", index=False)

# Display scores for verification
# print(score_df[1])


Evaluating:   0%|          | 0/350 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[119]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-NECJ9oQkme19vSkrmpB3mv6s on tokens per min (TPM): Limit 200000, Used 197555, Requested 3450. Please try again in 301ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[164]: TimeoutError()


In [ ]:
score_df[['semantic_similarity']].mean(axis=0)

score_df[['faithfulness','answer_relevancy', 'context_precision', 'context_recall',
       'context_entity_recall', 'semantic_similarity', 'answer_correctness',]].mean(axis=0)

,0
faithfulness,0.928401
answer_relevancy,0.717284
context_precision,0.627222
context_recall,0.612245
context_entity_recall,0.087333
semantic_similarity,0.835540
answer_correctness,0.491422
